In [ ]:
# -*- coding: utf-8 -*-
"""
Spyder Editor
This is a temporary script file.
"""
import gensim
from gensim.corpora.wikicorpus import WikiCorpus
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from pprint import pprint
import multiprocessing
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

wiki = WikiCorpus("kowiki-20190101-pages-articles.xml.bz2")

In [2]:
class TaggedWikiDocument(object):
    def __init__(self, wiki):
        self.wiki = wiki
        self.wiki.metadata = True
    def __iter__(self):
        for content, (page_id, title) in self.wiki.get_texts():
            yield TaggedDocument([c for c in content], [title])

In [ ]:
documents = TaggedWikiDocument(wiki)

cores = multiprocessing.cpu_count() * 2

model = Doc2Vec(dm=1, dm_mean=1, size=200, window=8, min_count=19, epochs=10, workers=cores)
  
model.build_vocab(documents)
print(str(model))

model.train(documents, total_examples=model.corpus_count, epochs=model.epochs)


Doc2Vec(dm/m,d200,n5,w8,mc19,s0.001,t8)


In [ ]:
from scipy.spatial import distance

def doc_distance(a,b):
    return distance.euclidean(model.docvecs[a], model.docvecs[b])


with open('kowiki_titles') as f:
    content = f.readlines()

content = [x.strip() for x in content] 

In [ ]:
import mysql.connector as conn

db=conn.connect(host='ls-0f21b16519d9a5d94fae17a404982290420e3981.ce70zohzx2qc.ap-northeast-2.rds.amazonaws.com', user='dbmasteruser', passwd='encore365!', database='WIKI')


In [ ]:
def insert(title, similar):
    cursor = db.cursor()    
    cursor.execute('INSERT INTO SIMILARS(TITLE, SIMILAR) VALUES(%s,%s)', (title, similar))


for line in content:
    title = line.strip()
    try:
        similars = "\t".join([x[0] for x in model.docvecs.most_similar(positive=[title], topn=10)])
        insert(title, similars)
    except:
        pass
